<h1>INDIAN FOOD ANALYSIS</h1>

<center><img src= "https://imgmediagumlet.lbb.in/media/2019/04/5cbc5ded512c0c3e8b6c782d_1555848685957.jpg?fm=webp&w=750&h=500&dpr=1" alt ="Food" align='center' style='width: 300px; height:400px;'></center>
                                                                                                               

<p>The food data set contain various information about different Indian cuisines.Through the following analysis, i have tried to discover some interesting insights to get better understanding of the Indian food</p>

<h2>Table of Content</h2>
<ol>
    <li><a href="#import">Importing Libraries and data</a></li>
    <li><a href="#describe">Understanding the data</a></li>
    <li><a href="#cleaning">Cleaning the data</a></li>
    <li><a href="#visual">Visualizing the data</a>
        <ol>
            <li><a href="#diet">Percentage of veg and non-veg overall,across regions and in different flavor_profile</a></li>
            <li><a href="#flavour">Preference of flavor among different regions through heatmap</a></li>
            <li><a href="#time">Dishes with highest prepration time and cooking time</a></li>
            <li><a href="#course">Overview of prep time and cook time among different course through boxplot</a></li>
            <li><a href="#ingredients">Total Variety of ingredients used in different state and most prefered ingredients across states</a></li>
            <li><a href="#dish">Word Cloud of dishes across regions</a></li>
        </ol>
    </li>
</ol>

<h2><span id="import">Importing Libraries and data</span></h2>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import collections
import json
import plotly.graph_objects as go
from wordcloud import WordCloud
plt.style.use('ggplot')
df=pd.read_csv('/kaggle/input/indian-food-101/indian_food.csv')
df.head()

<h2><span id="describe">Understanding the data</span></h2>
NOTE: The presence of -1 in any column shows the absence of data

In [ ]:
df.describe() #summary of numerical columns

The above statistics show that the average prep_time and cook_time of dishes is around 31 and 35 minutes whereas the maximum time are 500 and 720 minutes.So it seems that there are oultiers in these columns,we will check that later in the notebook through boxplot.

In [ ]:
df.describe(exclude=np.number) #summary of non-numerical columns

The above stats show that majority of the indian diet is vegeterian and includes spicy flavours.It also shows that the dataset contain a large amount of dishes from west region(state='Gujarat')

In [ ]:
df.info()

In [ ]:
df.dtypes

In [ ]:
df.isna().sum() #shows how many null values are present in each column

In [ ]:
df['region'].unique()

As we can see the region column contains null value,so firstly we will convert the null value to -1 to denote all the absent data with same value.

In [ ]:
df['region'].replace(np.nan,'-1',inplace=True)

Now,we will check for the columns where data is absent

In [ ]:
df[df['region']=='-1']['name'].values  #shows the name of food for which the region is unknown

In [ ]:
df[df['state']=='-1']['name'].values

In [ ]:
df[df['flavor_profile']=='-1']['name'].values

In [ ]:
df[df['prep_time']==-1.0]['name'].values

In [ ]:
df[df['cook_time']==-1.0]['name'].values

<h2><span id="cleaning">Cleaning the data</span></h2>
<ol>
    <li>For the prep_time and cook_time,i am replacing -1 values with the median of the column data</li>
    <li>For state,region and flavor_profile,i am replacing -1 values with 'unknown'</li>
</ol>

In [ ]:
mean_prep_time=df['prep_time'].median()
df['prep_time'].replace(-1.0,mean_prep_time,inplace=True)
mean_cook_time=df['cook_time'].median()
df['cook_time'].replace(-1.0,mean_cook_time,inplace=True)

In [ ]:
df['region'].replace('-1','unknown',inplace=True)
df['state'].replace('-1','unknown',inplace=True)
df['flavor_profile'].replace('-1','unknown',inplace=True)

<h2><span id="visual">Visualizing the data</span></h2>

<h3><span id="diet">Percentage of veg and non-veg overall,across regions and in different flavor_profile </span></h3>

In [ ]:
diet_type=df['diet'].value_counts()
fig=px.pie(values=diet_type,names=diet_type.keys(),title='Percentage of veg and non-veg dishes')
fig.show()

In [ ]:
fig=px.histogram(df,x='flavor_profile',color='diet',color_discrete_sequence=px.colors.sequential.Blackbody,title='Amount of veg and non-veg dishes in different flavours and overall flavour_profile count among dishes')
fig.update_layout(yaxis_title='amount_of_dishes')

In [ ]:
fig=px.histogram(df,x='region',color='diet',color_discrete_sequence=px.colors.sequential.Rainbow,title='Amount of veg and non-veg dishes across different regions')
fig.update_layout(yaxis_title='amount_of_dishes')

The above graph shows that all regions except North east have majority of the dishes vegeterian

<h3><span id="flavour">Preference of flavor among different regions through heatmap</span></h3>

In [ ]:
df1=df[['region','flavor_profile']]
df_group=df1.groupby(['region','flavor_profile']).size().reset_index(name='count')
px.density_heatmap(df_group,x='region',y='flavor_profile',z='count')

It is clear form the above map that almost all regions prefer spicy flavours over others.However,the East region has a strong inclination towards sweet flavour.

<h3><span id="time">Dishes with highest prepration time and cooking time</span></h3>

In [ ]:
df2=df.sort_values(by='prep_time',ascending=False)[:5]
px.bar(df2,x='name',y='prep_time',color_discrete_sequence=px.colors.sequential.Agsunset,color='state',title='Top 5 dishes with highest prep_time and the state to which the dish belong')

In [ ]:
df3=df.sort_values(by='cook_time',ascending=False)[:5]
px.bar(df3,x='name',y='cook_time',color='state',color_discrete_sequence=px.colors.sequential.Emrld,title='Top 5 dishes with highest cook_time and the state to which the dish belong')

<h3><span id="course">Overview of prep time and cook time among different course through boxplot</span></h3>

In [ ]:
px.box(df,x='course',y='prep_time',color_discrete_sequence=px.colors.sequential.Electric,color='diet')

In [ ]:
px.box(df,x='course',y='cook_time',color_discrete_sequence=px.colors.sequential.Electric,color='diet')

It is clear from the above two plots that there are outliers present in the column prep_time and cook_time.It seems that snacks have higher prep_time than other courses on average while desserts have higher cook_time over others. Although it seems that starters have the highest prep_time and cook_time but actually there is only two starter dishes in the data and both are non-vegeterian,so it is hard to say much about starters average prep_time and cook_time from the given data.

<h3><span id="ingredients">Total Variety of ingredients used in different state and most prefered ingredients across states</span></h3>

In [ ]:
df_state=df[['state','ingredients']]
temp=df_state.groupby(['state']).agg({'ingredients':','.join}).reset_index()
d=[]
for i in temp['ingredients']:
    i=i.split(',')
    new_list = [x.strip() for x in i]
    dic=collections.Counter(new_list)
    d.append(list(collections.OrderedDict(dic).keys())[:4])
ingredient_list=pd.DataFrame()
ingredient_list['State']=temp['state']
ingredient_list['Most_preferred_ingredients']=d
fig=go.Figure(data=[go.Table(header=dict(values=ingredient_list.columns,align='left',font=dict(color='black',size=14),line_color='darkslategray',height=30),
                    cells=dict(values=[ingredient_list['State'],ingredient_list['Most_preferred_ingredients']],align='left',line_color='darkslategray',height=40))])
fig.show()

In [ ]:
#temp data frame contains ingredients grouped by state
len_col=[]
for i in temp['ingredients']:
    len_col.append(len(set(i.split(','))))
len_amt=pd.DataFrame()
len_amt['total_ingredients']=len_col
len_amt['state']=temp['state']

In [ ]:
india_states=json.load(open('/kaggle/input/state-india/states_india.geojson','r'))
state_id_map = {}
for feature in india_states["features"]:
    feature["id"] = feature["properties"]["state_code"]
    state_id_map[feature["properties"]["st_nm"]] = feature["id"]
len_amt["id"] = len_amt['state'][:24].apply(lambda x: state_id_map[x])

In [ ]:
fig=px.choropleth(len_amt,locations='id',geojson=india_states,color='total_ingredients',scope='asia',
                  hover_name='state',title='Total Variety of ingredients across different states of india')
fig.update_geos(fitbounds="locations",visible=False)
fig.show()

The map shows that there is a large variety of ingredients used in Gujrat,Punjab and Mahrashtra.That suggest that there is a large variety of cuisines available in these states. 
Although the above result may be correct,but i am not sure that the dataset contain all the cuisines avaliable in all the states.Since India has a rich variety of flavours and every place has some of their own unique ingredients,so there is a possibility that the results might be slightly different if there would be more data.But again i'm not sure about it.So,If anyone knows about this thing then please tell me in the comment section.

In [ ]:
total_amt_ingredients=[]
for i in df['ingredients']:
    total_amt_ingredients.append(len(i.split(',')))

min_ingredient_index=total_amt_ingredients.index(min(total_amt_ingredients))
min_ingredient_name=[df['name'][min_ingredient_index]]
max_ingredient_index=total_amt_ingredients.index(max(total_amt_ingredients))
max_ingredient_name=[df['name'][max_ingredient_index]]

fig=go.Figure(data=go.Bar(x=[min_ingredient_name[0],max_ingredient_name[0]],y=[min(total_amt_ingredients),max(total_amt_ingredients)]))
fig.update_layout(title='Dishes having maximum and minimum amount of ingredients',yaxis_title='amount of ingredients',xaxis_title='dish_name')


<h3><span id="dish">Word Cloud of dishes across regions</span></h3>

The word cloud shows the name of popular Indian cuisines across different regions

In [ ]:
def word_cloud(region,color):
    text=' '.join(df[df['region']==region]['name'])
    word_cloud = WordCloud(background_color=color,max_font_size=50,max_words=50).generate(text)
    plt.figure(figsize=(15,7))
    plt.axis("off")
    plt.imshow(word_cloud,interpolation='bilinear')
    return plt.show()

In [ ]:
word_cloud('West','#A52A2A')

In [ ]:
word_cloud('East','#D1B26F')

In [ ]:
word_cloud('North','white')

In [ ]:
word_cloud('South','#DA5200')

In [ ]:
word_cloud('North East','#DC143C')

If you find any error or improvement in my solution,then please do tell me in the comments.
<b>All feedback are appreciated!</b><br>
And If you like my work, then please upvote it.
<b>Thank You in advance!!</b>